# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, в каком регионе бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data0.info()
data1.info()
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

Открываем файл с данными и изучаем общую информацию. В таблицах нет пропусков. Большинство данных имеют тип  *float, object*.

Округлим вверх объём запасов в скважине (тыс. баррелей), обратимся к функции *np.ceil()*  и сделаем тип данных целочисленным с помощью метода *astype()*.

In [4]:
data0['product'] = np.ceil(data0['product']).astype('int')
data1['product'] = np.ceil(data1['product']).astype('int')
data2['product'] = np.ceil(data2['product']).astype('int')

Исключим категориальный признак *'id'*  в каждой таблице, поскольку он портит модель.

In [5]:
del data0['id']
del data1['id']
del data2['id']

In [6]:
print('Количество дубликатов в таблице data0: ', data0.duplicated().sum())
print('Количество дубликатов в таблице data1: ', data1.duplicated().sum())
print('Количество дубликатов в таблице data2: ', data2.duplicated().sum())

Количество дубликатов в таблице data0:  0
Количество дубликатов в таблице data1:  0
Количество дубликатов в таблице data2:  0


Дубликатов нет.

## Обучение и проверка модели

Для того, чтобы разбить данные на выборки воспользуемся методом *train_test_split()*. В обучающей от всего набора данных получилось 75%, 25% - в валидационной.

In [7]:
def split_and_train(data):
    
    features = data.drop(['product'], axis=1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=12345)
    scaler = MinMaxScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    return features_train, features_valid, target_train, target_valid.reset_index(drop=True)

In [8]:
features0_train, features0_valid, target0_train, target0_valid = split_and_train(data0)

In [9]:
features1_train, features1_valid, target1_train, target1_valid = split_and_train(data1)

In [10]:
features2_train, features2_valid, target2_train, target2_valid = split_and_train(data2)

Наиболее распространённая метрика качества в задаче регрессии — средняя квадратичная ошибка, MSE. Величина MSE должна быть как можно меньше. Квадратные единицы нам не нужны, поэтому возьмем корень из этой величины и получим величину RMSE.

In [11]:
def modelLR(features_train,target_train,features_valid,target_valid):
    model = LinearRegression(normalize  = False)
    model.fit(features_train,target_train) # обучаем модель на тренировочной выборке
    predictions_valid = model.predict(features_valid) # получаем предсказания модели на валидационной выборке

    predictions_valid = pd.Series(predictions_valid)
    
    result = mean_squared_error(target_valid, predictions_valid)**0.5 # посчитаем значение метрики RMSE на валидационной выборке
    print("RMSE модели линейной регрессии на валидационной выборке: ", result)
    mean_volue = round(predictions_valid.mean(),2)
    print("Cредний запас предсказанного сырья: ", mean_volue)
    
    return  predictions_valid

In [12]:
print("0-ой регион:")
predictions0_valid = modelLR(features0_train,target0_train,features0_valid,target0_valid)

0-ой регион:
RMSE модели линейной регрессии на валидационной выборке:  37.58106084247383
Cредний запас предсказанного сырья:  93.09


In [13]:
print("1-ой регион:")
predictions1_valid = modelLR(features1_train,target1_train,features1_valid,target1_valid)

1-ой регион:
RMSE модели линейной регрессии на валидационной выборке:  1.0627821385747935
Cредний запас предсказанного сырья:  69.09


In [14]:
print("2-ой регион:")
predictions2_valid = modelLR(features2_train,target2_train,features2_valid,target2_valid)

2-ой регион:
RMSE модели линейной регрессии на валидационной выборке:  40.03083681476167
Cредний запас предсказанного сырья:  95.47


## Подготовка к расчёту прибыли

Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [15]:
income_per_bar = 450*10**3
budget = 10**10

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Для этого необходимо взять отношение бюджета на разработку скважин в регионе и доход с каждой единицы продукта.

In [16]:
print("Достаточный объём сырья для безубыточной разработки новой скважины  в тыс. баррелях: ", int(budget  / income_per_bar))

Достаточный объём сырья для безубыточной разработки новой скважины  в тыс. баррелях:  22222


Сравним полученный объём сырья со средним запасом в каждом регионе.

In [17]:
print("Средний запас нефти в скважине для безубыточного функционирования: ",np.ceil(int(budget  / income_per_bar)  / 200))

Средний запас нефти в скважине для безубыточного функционирования:  112.0


Средний запас нефти в скважине для безубыточного функционирования больше, чем среднее значение запаса сырья в каждом регионе. 

## Расчёт прибыли и рисков 

Напишем функцию *revenue()* для расчёта прибыли по выбранным скважинам и предсказаниям модели:
 - выберите скважины с максимальными значениями предсказаний (top_probs);
 - просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям (revenue);
 - рассчитайте прибыль для полученного объёма сырья (revenue - budget).

In [18]:
def revenue(predictions, target, count):
    top_preds = predictions.sort_values(ascending=False)
    top_target = target[top_preds.index][:count]
    revenue = top_target.sum() * income_per_bar
    return revenue - budget

Посчитаем риски и прибыль для каждого региона.

In [19]:
state = np.random.RandomState(12345)

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [20]:
def interval_and_risk(predictions, target):
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
    
        values.append(revenue(preds_subsample,target_subsample,200))

    values = pd.Series(values)
    mean_revenue = int(values.mean())
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk = st.percentileofscore(values, 0)

    return (lower, upper), mean_revenue, risk

In [21]:
interval_and_risk(predictions0_valid, target0_valid)

((-55607499.99999999, 993556249.9999999), 470707550, 4.4)

In [22]:
interval_and_risk(predictions1_valid, target1_valid)

((155071250.0, 967422500.0), 540547100, 0.1)

In [23]:
interval_and_risk(predictions2_valid, target2_valid)

((-68567500.0, 1032706250.0), 465326450, 4.7)

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. У нас получилось, что это первый регион. Следовательно, первый регион - это регион, где добыча принесёт наибольшую прибыль и риски минимальны.